In [4]:
import cv2
import numpy as np
from mtcnn import MTCNN

# Load the MTCNN detector
detector = MTCNN()

# Set up the video capture device
cap = cv2.VideoCapture(0)

# Define the function to estimate gaze direction
def estimate_gaze(eye_left, eye_right):
    # Calculate the center of the eyes
    eye_left_center = np.mean(eye_left, axis=0).astype(int)
    eye_right_center = np.mean(eye_right, axis=0).astype(int)
    
    # Calculate the horizontal and vertical distances between the eyes
    eye_dist_horizontal = eye_right_center[0] - eye_left_center[0]
    eye_dist_vertical = eye_right_center[1] - eye_left_center[1]
    
    # Calculate the angle of the gaze vector in radians
    gaze_angle_horizontal = np.arctan(eye_dist_horizontal / eye_dist_vertical)
    gaze_angle_vertical = np.arctan(eye_dist_vertical / eye_dist_horizontal)
    
    return gaze_angle_horizontal, gaze_angle_vertical

# Start the video feed and loop over the frames
while True:
    # Read a frame from the video feed
    ret, frame = cap.read()
    
    # Use the MTCNN detector to detect the face and eyes in the frame
    result = detector.detect_faces(frame)
    if len(result) > 0:
        # Extract the coordinates of the eyes from the face detection result
        bounding_box = result[0]['box']
        keypoints = result[0]['keypoints']
        eye_left = keypoints['left_eye']
        eye_right = keypoints['right_eye']
        
        # Draw rectangles around the eyes and face
        cv2.rectangle(frame, (bounding_box[0], bounding_box[1]), (bounding_box[0]+bounding_box[2], bounding_box[1]+bounding_box[3]), (0, 255, 0), 2)
        cv2.rectangle(frame, (eye_left[0]-5, eye_left[1]-5), (eye_left[0]+5, eye_left[1]+5), (0, 0, 255), 2)
        cv2.rectangle(frame, (eye_right[0]-5, eye_right[1]-5), (eye_right[0]+5, eye_right[1]+5), (0, 0, 255), 2)
        
        # Estimate the gaze direction
        gaze_angle_horizontal, gaze_angle_vertical = estimate_gaze(eye_left, eye_right)
        
        # Display the gaze direction on the screen
        cv2.putText(frame, f'Gaze direction: H={gaze_angle_horizontal:.2f}, V={gaze_angle_vertical:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    
    # Display the resulting frame
    cv2.imshow('frame', frame)
    
    # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture device and close the windows
cap.release()
cv2.destroyAllWindows()


2023-04-12 15:10:33.179544: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 15:10:33.423092: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-12 15:10:34.054386: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/yramakrishna/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:
2023-04-12 15:10:34.054436: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic l

1/1 [==============================] - 0s 118ms/step


IndexError: invalid index to scalar variable.

In [5]:
import pygame.camera
import pygame.surfarray
import numpy as np

# initialize Pygame and camera
pygame.init()
pygame.camera.init()
camera = pygame.camera.Camera(pygame.camera.list_cameras()[0])
camera.start()

# set up display and clock
display_size = (1080, 720)
display = pygame.display.set_mode(display_size)
pygame.display.set_caption("Pygame Webcam Capture")
clock = pygame.time.Clock()

# loop through frames until user exits
while True:
    # capture frame from camera
    frame = camera.get_image()

    # convert frame to a numpy array
    frame_array = pygame.surfarray.array3d(frame)

    # display the captured frame
    pygame.surfarray.blit_array(display, frame_array)
    pygame.display.update()

    # calculate the frame rate and time elapsed
    fps = int(clock.get_fps())
    time_elapsed = int(pygame.time.get_ticks() / 1000)

    # display the frame rate and time elapsed
    font = pygame.font.SysFont('Arial', 20)
    fps_text = font.render(f"FPS: {fps}", True, (255, 255, 255))
    time_text = font.render(f"Time Elapsed: {time_elapsed}s", True, (255, 255, 255))
    display.blit(fps_text, (10, 10))
    display.blit(time_text, (10, 30))

    # handle events and wait for a key press to exit
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_q):
            camera.stop()
            pygame.quit()
            quit()

    # tick the clock
    clock.tick(30)

pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemError: ioctl(VIDIOC_S_FMT) failure: no supported formats

In [2]:
import cv2
import time

# create a VideoCapture object to capture video from default camera
cap = cv2.VideoCapture(0)

# get the frame width and height
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# initialize variables for frame rate calculation
prev_time = 0
frame_counter = 0

# loop through frames until user exits
while True:
    # read a frame from the video capture
    ret, frame = cap.read()

    if ret:
        # calculate the frame rate
        current_time = time.time()
        time_elapsed = current_time - prev_time
        prev_time = current_time
        fps = int(frame_counter / time_elapsed)
        frame_counter += 1

        # display the frame rate and time elapsed
        cv2.putText(frame, f"FPS: {fps}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        cv2.putText(frame, f"Time Elapsed: {int(time_elapsed)}s", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # display the captured frame
        cv2.imshow('Video Capture', frame)

    # wait for a key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release the video capture and destroy the window
cap.release()
cv2.destroyAllWindows()


QObject::moveToThread: Current thread (0xddcb80) is not the object's thread (0x1ca84b0).
Cannot move to target thread (0xddcb80)

QObject::moveToThread: Current thread (0xddcb80) is not the object's thread (0x1ca84b0).
Cannot move to target thread (0xddcb80)

QObject::moveToThread: Current thread (0xddcb80) is not the object's thread (0x1ca84b0).
Cannot move to target thread (0xddcb80)

QObject::moveToThread: Current thread (0xddcb80) is not the object's thread (0x1ca84b0).
Cannot move to target thread (0xddcb80)

QObject::moveToThread: Current thread (0xddcb80) is not the object's thread (0x1ca84b0).
Cannot move to target thread (0xddcb80)

QObject::moveToThread: Current thread (0xddcb80) is not the object's thread (0x1ca84b0).
Cannot move to target thread (0xddcb80)

QObject::moveToThread: Current thread (0xddcb80) is not the object's thread (0x1ca84b0).
Cannot move to target thread (0xddcb80)

QObject::moveToThread: Current thread (0xddcb80) is not the object's thread (0x1ca84b0).
C

In [6]:
import pygame
import cv2

# Set up the Pygame window
WINDOW_SIZE = (640, 480)
pygame.init()
screen = pygame.display.set_mode(WINDOW_SIZE)
pygame.display.set_caption("Pygame Webcam")

# Set up the video capture
cap = cv2.VideoCapture(0)

# Set up the clock and frame count
clock = pygame.time.Clock()
frame_count = 0

while True:
    # Handle events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            cap.release()
            pygame.quit()
            quit()

    # Read a frame from the video capture
    ret, frame = cap.read()

    # Convert the frame to a Pygame surface
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.flip(frame, 1)
    frame_surface = pygame.surfarray.make_surface(frame)

    # Display the frame surface on the screen
    screen.blit(frame_surface, (0, 0))

    # Update the frame count and calculate the frame rate
    frame_count += 1
    elapsed_time = pygame.time.get_ticks() / 1000
    frame_rate = frame_count / elapsed_time

    # Display the frame rate and elapsed time in the window title
    pygame.display.set_caption(f"Pygame Webcam ({frame_rate:.1f} fps, {elapsed_time:.1f} s)")

    # Update the display and limit the frame rate to 30 FPS
    pygame.display.update()
    clock.tick(30)


libEGL warning: DRI3: Screen seems not DRI3 capable
libEGL warning: DRI2: failed to authenticate
libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

[ WARN:0@1129.528] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@1129.528] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [9]:
import cv2

# create a VideoCapture object to capture video from default camera
cap = cv2.VideoCapture(0)

# loop through frames until user exits
while True:
    # read a frame from the video capture
    ret, frame = cap.read()

    # display the captured frame
    cv2.imshow('Video Capture', frame)

    # wait for a key press to exitq
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release the video capture and destroy the window
cap.release()
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x28ced70) is not the object's thread (0x21d0d30).
Cannot move to target thread (0x28ced70)

QObject::moveToThread: Current thread (0x28ced70) is not the object's thread (0x21d0d30).
Cannot move to target thread (0x28ced70)

QObject::moveToThread: Current thread (0x28ced70) is not the object's thread (0x21d0d30).
Cannot move to target thread (0x28ced70)

QObject::moveToThread: Current thread (0x28ced70) is not the object's thread (0x21d0d30).
Cannot move to target thread (0x28ced70)

QObject::moveToThread: Current thread (0x28ced70) is not the object's thread (0x21d0d30).
Cannot move to target thread (0x28ced70)

QObject::moveToThread: Current thread (0x28ced70) is not the object's thread (0x21d0d30).
Cannot move to target thread (0x28ced70)

QObject::moveToThread: Current thread (0x28ced70) is not the object's thread (0x21d0d30).
Cannot move to target thread (0x28ced70)

QObject::moveToThread: Current thread (0x28ced70) is not the object's thread

In [4]:
import cv2
from openface import OpenFaceGaze

# Initialize the OpenFace gaze model
gaze_model = OpenFaceGaze()

# Open a video capture device (e.g. webcam)
cap = cv2.VideoCapture(0)

# Loop over video frames
while True:
    # Capture a frame from the video device
    ret, frame = cap.read()

    # Check if the frame was successfully captured
    if not ret:
        break

    # Get the gaze direction from the OpenFace gaze model
    gaze_direction = gaze_model.get_gaze(frame)

    # Display the gaze direction on the frame
    cv2.putText(frame, "Gaze direction: {}".format(gaze_direction), (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame with the gaze direction
    cv2.imshow('frame', frame)

    # Exit on key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture device and close all windows
cap.release()
cv2.destroyAllWindows()

ImportError: cannot import name 'OpenFaceGaze' from 'openface' (/home/yramakrishna/anaconda3/lib/python3.9/site-packages/openface/__init__.py)

In [ ]:
import pygame
import cv2
import time

# Initialize Pygame
pygame.init()

# Set the width and height of the window
width, height = 640, 480

# Set the capture device and the desired frame size
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

# Create the Pygame display surface
screen = pygame.display.set_mode((width, height))

# Set the Pygame clock
clock = pygame.time.Clock()

# Initialize the frame counter and start time
frame_count = 0
start_time = time.time()

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()

    # Convert the frame to a Pygame surface
    surface = pygame.surfarray.make_surface(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Draw the surface onto the display surface
    screen.blit(surface, (0, 0))

    # Increment the frame counter and calculate the elapsed time
    frame_count += 1
    elapsed_time = time.time() - start_time

    # Calculate the frame rate and display it on the Pygame window
    if elapsed_time > 1:
        frame_rate = frame_count / elapsed_time
        frame_count = 0
        start_time = time.time()
        pygame.display.set_caption(f'Video Capture (FPS: {round(frame_rate, 2)})')

    # Update the Pygame display
    pygame.display.flip()

    # Handle Pygame events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            # Release the video capture and close the Pygame window
            cap.release()
            pygame.quit()
            quit()
            break

# Release the video capture and close the Pygame window
cap.release()
pygame.quit()
quit()


pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
# import cv2
# cv2.__version__